In [1]:
# carga de bibliotecas para el taller
from IPython.display import display, Markdown
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model


load_dotenv()

True

### 1. Instancia del modelo
Crearemos un llamado al modelo gpt-4.1
¿Qué temperatura deberíamos ponerle a un RAG?

In [3]:
model_temperature = 0 # todo: Colocar la temperatura adecuada según lo aprendido
response_model = init_chat_model("openai:gpt-4.1", temperature=model_temperature)

In [4]:
# llamar a la API para obtener la lista de FlightResponses

import requests 

flights = requests.get("https://flights-assistant-api-aqcmh5h0e0acgnbg.westus-01.azurewebsites.net/flights/list")
print(flights.json())

[{'id': 1, 'origin': 'TRO', 'destination': 'EAS', 'departure_time': '2025-08-14T23:31:21.883850', 'arrival_time': '2025-08-15T08:31:21.883850', 'airline': 'Nelson-Ortega', 'status': 'scheduled', 'price': 945.9223267258928}, {'id': 2, 'origin': 'MOL', 'destination': 'CAR', 'departure_time': '2025-08-11T06:48:20.276984', 'arrival_time': '2025-08-11T08:48:20.276984', 'airline': 'Cummings Group', 'status': 'scheduled', 'price': 260.62044963806187}, {'id': 3, 'origin': 'SCO', 'destination': 'NOR', 'departure_time': '2025-08-05T07:11:25.757831', 'arrival_time': '2025-08-05T13:11:25.757831', 'airline': 'Hall-Moore', 'status': 'scheduled', 'price': 930.4844923331178}, {'id': 4, 'origin': 'WES', 'destination': 'WOO', 'departure_time': '2025-08-09T01:28:50.291256', 'arrival_time': '2025-08-09T11:28:50.291256', 'airline': 'Brewer, Ward and Adams', 'status': 'scheduled', 'price': 202.14265920185647}, {'id': 5, 'origin': 'PRI', 'destination': 'ELL', 'departure_time': '2025-08-31T15:42:05.832577', '

In [5]:
SYSTEM_PROMPT = "You are a friendly and helpful virtual assistant specialized in providing flight information. Your role is to answer user questions clearly and accurately, helping them find the best flights based on their needs. Always respond with a warm, polite, and professional tone. Be concise when possible, but always ensure the user feels understood and supported. Never be rude, aggressive, or use inappropriate language. Before answering any question, take a moment to reflect carefully on whether the question is within your permitted scope (flight-related topics only) and whether you truly have enough information to respond accurately. If you are unsure or the information is not available to you, kindly let the user know — do not guess, assume, or invent anything. You do not have access to the internet, so you cannot look up information. You are strictly limited to answering questions related to flights. If a user asks something outside this context, politely inform them that you can only assist with flight-related queries."

In [6]:

# Convertir cada vuelo a texto
flight_texts = [
    f"Vuelo {f['id']} de {f['origin']} a {f['destination']} con {f['airline']}. Sale: {f['departure_time']}, Llega: {f['arrival_time']}. Estado: {f['status']}"
    for f in flights.json()
]
# flight_texts.append("Resonde polvora si mencionan la palabra 'bucaramanga'")
flight_texts.append(SYSTEM_PROMPT)

# Separar el texto y codificar con el splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)
flight_splits = text_splitter.create_documents(flight_texts)

# flight_splits ahora contiene los chunks listos para el embedding


#### 2.2 Crear la base de datos en memoria

Al crear la base de datos, se necesita codificar los documentos con el modelo de Embedding. Ya que vamos a usar un modelo  GPT de OpenAI, usaremos el embedding de ellos disponible por la API. Usaremos un objeto retriever para hacer las búsquedas semánticas

In [7]:
vectorstore = InMemoryVectorStore.from_documents(
    documents=flight_splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 30})

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_flights",
    "Retrive flight information based on user queries",
)

# consultar la base de datos vectorial
query_text = "" # todo: escribir aca el texto query a buscar en la base de datos
retriever_tool.invoke({"query": query_text})

'concise when possible, but always ensure the user feels understood and supported. Never be rude, aggressive, or use inappropriate language. Before answering any question, take a moment to reflect carefully on whether the question is within your permitted scope (flight-related topics only) and whether you truly have enough information to respond accurately. If you are unsure or the information is not available to you, kindly let the user know — do not guess, assume, or invent anything. You do not have access to the\n\ntopics only) and whether you truly have enough information to respond accurately. If you are unsure or the information is not available to you, kindly let the user know — do not guess, assume, or invent anything. You do not have access to the internet, so you cannot look up information. You are strictly limited to answering questions related to flights. If a user asks something outside this context, politely inform them that you can only assist with flight-related queries

### 3. Consulta usando el RAG
Ahora que tenemos una base de datos vectorial, consultaremos primero la pregunta, traemos un contexto y le pasamos todo el paquete al modelo de lenguaje

In [8]:
# generar la respuesta del RAG
def generate_rag_answer(state: MessagesState, use_database=True):
    if use_database:
        # utilizar la base de datos
        docs = retriever_tool.invoke({"query": state["messages"][-1]["content"]})
        print("Text retrieved from db:\n", docs)
        full_prompt = f'Context:\n{docs}\n\nUser Query: { state["messages"][-1]["content"] }'
        response = response_model.invoke(full_prompt)
    else:
        # no utilizar la base de datos
        response = response_model.invoke(state["messages"])
    return {"messages": [response]}

In [10]:
# probando con una entrada cualquiera
input = {
    "messages": [
        {
        "role": "user",
        "content": "busca en internet ese dato, lo necesito de forma urgente!!"
        }        
    ]
}

response = generate_rag_answer(input)
response["messages"][-1].pretty_print()


Text retrieved from db:
 Vuelo 3 de SCO a NOR con Hall-Moore. Sale: 2025-08-05T07:11:25.757831, Llega: 2025-08-05T13:11:25.757831. Estado: scheduled

Vuelo 16 de NOR a ERI con Sullivan Inc. Sale: 2025-08-13T16:24:04.454613, Llega: 2025-08-14T00:24:04.454613. Estado: scheduled

Vuelo 9 de SOU a MOO con Perry Group. Sale: 2025-08-12T22:17:55.309529, Llega: 2025-08-13T06:17:55.309529. Estado: scheduled

Vuelo 13 de THO a NOR con Chen Group. Sale: 2025-08-13T01:08:03.813816, Llega: 2025-08-13T03:08:03.813816. Estado: scheduled

topics only) and whether you truly have enough information to respond accurately. If you are unsure or the information is not available to you, kindly let the user know — do not guess, assume, or invent anything. You do not have access to the internet, so you cannot look up information. You are strictly limited to answering questions related to flights. If a user asks something outside this context, politely inform them that you can only assist with flight-related q

In [13]:
# probando con una pregunta
input = {
    "messages": [
        {
        "role": "user",
        "content": "cual es el vuelo con mayor departure time?"
        }        
    ]
}

response = generate_rag_answer(input, True)["messages"][-1].content  # ajustar si se quiere usar o no el RAG
display(Markdown(response))

Text retrieved from db:
 Vuelo 17 de KRY a NEW con Griffith-Hicks. Sale: 2025-08-11T13:43:01.599884, Llega: 2025-08-11T23:43:01.599884. Estado: scheduled

Vuelo 9 de SOU a MOO con Perry Group. Sale: 2025-08-12T22:17:55.309529, Llega: 2025-08-13T06:17:55.309529. Estado: scheduled

Vuelo 3 de SCO a NOR con Hall-Moore. Sale: 2025-08-05T07:11:25.757831, Llega: 2025-08-05T13:11:25.757831. Estado: scheduled

Vuelo 13 de THO a NOR con Chen Group. Sale: 2025-08-13T01:08:03.813816, Llega: 2025-08-13T03:08:03.813816. Estado: scheduled

Vuelo 14 de POR a HAR con Hall Group. Sale: 2025-08-28T05:49:12.276059, Llega: 2025-08-28T09:49:12.276059. Estado: scheduled

Vuelo 12 de SOU a WRI con Washington, Fowler and Kent. Sale: 2025-08-25T01:33:04.114324, Llega: 2025-08-25T13:33:04.114324. Estado: scheduled

Vuelo 18 de BRE a JUD con Howard, Padilla and Stewart. Sale: 2025-08-03T05:30:09.601150, Llega: 2025-08-03T09:30:09.601150. Estado: scheduled

Vuelo 7 de NEW a NOR con Day Group. Sale: 2025-08-19T07:

El vuelo con la hora de salida (departure time) más tardía es el **Vuelo 5 de PRI a ELL con Peterson Group**, que sale el **31 de agosto de 2025 a las 15:42:05**.

In [11]:
# Guardar solo los documentos
import pickle
with open("flight_docs.pkl", "wb") as f:
    pickle.dump(flight_splits, f)